<img src="https://github.com/Christina1281995/demo-repo/blob/main/header_absa1.png?raw=true" width="95%">

### What could be improved in this sentiment analysis?

<img src="https://github.com/Christina1281995/demo-repo/blob/main/limitsofSA.png?raw=true" width="75%">

##### Document Level

What we have been doing so far is called "document-level" analysis.

<img src="https://github.com/Christina1281995/demo-repo/blob/main/document level.png?raw=true" width="75%">

Let's take a look at the sentiment result in this tweet:

In [2]:
from transformers import pipeline

classifier = pipeline("text-classification")
classifier("This disease #covid19 is REALLY starting to annoy me, but at least the lockdown lets me spend more time with the family which has been amazing! Work-from-home is tough though...")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'NEGATIVE', 'score': 0.7361530661582947}]

##### Sentence Level

<img src="https://github.com/Christina1281995/demo-repo/blob/main/sentence level.png?raw=true" width="75%">

In [3]:
# Sentence 1:
classifier("This disease #covid19 is REALLY starting to annoy me, but at least the lockdown lets me spend more time with the family which has been amazing!")

[{'label': 'POSITIVE', 'score': 0.9957256317138672}]

In [4]:
# Sentence 2:
classifier("Work-from-home is tough though...")

[{'label': 'NEGATIVE', 'score': 0.994464099407196}]

##### Aspect Level

<img src="https://github.com/Christina1281995/demo-repo/blob/main/ABSA.png?raw=true" width="75%">